# Chapter 3. Effective Functions
## 3.1 Python’s Functions Are First-Class

In [88]:
import traceback as tb
from operator import itemgetter
import functools

In [3]:
def yell(text):
    return text.upper() + '!'

In [4]:
yell('hello')

'HELLO!'

In [5]:
bark = yell
bark('woof')

'WOOF!'

In [6]:
del yell
try:
    yell('hello')
except NameError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

NameError("name 'yell' is not defined")


  File "<ipython-input-6-5aec7493b02d>", line 3, in <module>
    yell('hello')


In [7]:
bark('hello')

'HELLO!'

In [8]:
bark.__name__

'yell'

### Functions Can Be Stored in Data Structures

In [9]:
funcs = [bark, str.lower, str.capitalize]
funcs

[<function __main__.yell(text)>,
 <method 'lower' of 'str' objects>,
 <method 'capitalize' of 'str' objects>]

In [10]:
s = 'hello'
for f in funcs:
    s = f(s)
s

'Hello!'

### Functions Can Be Passed to Other Functions

In [11]:
def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)

In [12]:
greet(bark)

HI, I AM A PYTHON PROGRAM!


In [13]:
for f in funcs:
    greet(f)

HI, I AM A PYTHON PROGRAM!
hi, i am a python program
Hi, i am a python program


In [14]:
def whisper(text):
    return text.lower() + '...'

greet(whisper)

hi, i am a python program...


In [15]:
list(map(bark, ['hello', 'hey', 'hi']))

['HELLO!', 'HEY!', 'HI!']

### Functions Can Be Nested

In [20]:
def speak(text):
    def whisper_inner(t):
        return t.lower() + '...'
    return whisper(text)

In [17]:
speak('Hello, World')

'hello, world...'

In [21]:
try:
    whisper_inner('Yo')
except NameError as e:
    print(repr(e))

NameError("name 'whisper_inner' is not defined")


In [22]:
try:
    speak.whisper
except AttributeError as e:
    print(repr(e))

AttributeError("'function' object has no attribute 'whisper'")


In [23]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    
    def yell(text):
        return text.upper() + '!'
    
    if volume > 0.5:
        return yell
    else:
        return whisper

In [25]:
get_speak_func(0.6)('say "wow!"')

'SAY "WOW!"!'

In [27]:
get_speak_func(0.4)('SHHH!')

'shhh!...'

### Functions Can Capture Local State

In [28]:
def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...'
    
    def yell():
        return text.upper() + '!'
    
    if volume > 0.5:
        return yell
    else:
        return whisper

In [29]:
get_speak_func('say "wow!"', 0.6)()

'SAY "WOW!"!'

In [30]:
def make_adder(n):
    def add(x):
        return x + n
    return add

In [31]:
make_adder(10)(5)

15

### Objects Can Behave Like Functions

In [32]:
class Adder:
    def __init__(self, n):
        self.n = n
    
    def __call__(self, x):
        return self.n + x

In [33]:
Adder(10)(5)

15

In [34]:
adder_1 = Adder(1)
adder_1(2)

3

In [35]:
callable(adder_1)

True

In [36]:
callable("Hello!")

False

## 3.2 Lambdas Are Single-Expression Functions

In [37]:
add = lambda x,y: x + y
add(3, 5)

8

In [38]:
(lambda x, y: x + y)(5, 3)

8

### Lambdas You Can Use

In [40]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
sorted(tuples)

[(1, 'd'), (2, 'b'), (3, 'c'), (4, 'a')]

In [41]:
sorted(tuples, key=lambda x: x[1])

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

In [45]:
list(map(itemgetter(1), tuples))

['d', 'b', 'a', 'c']

In [46]:
sorted(tuples, key=itemgetter(1))

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

In [42]:
sorted(range(-5, 6), key=lambda x: x * x)

[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]

In [47]:
def make_adder(n):
    return lambda x: x + n

In [48]:
make_adder(10)(5)

15

### But Maybe You Shouldn’t…

In [50]:
# Harmful:
list(filter(lambda x: x % 2 == 0, range(16)))

[0, 2, 4, 6, 8, 10, 12, 14]

In [51]:
# Better:
[x for x in range(16) if x % 2 == 0]

[0, 2, 4, 6, 8, 10, 12, 14]

## 3.3 The Power of Decorators
### Python Decorator Basics

In [52]:
def null_decorator(func):
    return func

In [53]:
@null_decorator
def greet():
    print('Hello!')

In [54]:
greet()

Hello!


### Decorators Can Modify Behavior

In [55]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    
    return wrapper

In [63]:
@uppercase
def greet():
    return 'Hello!'

In [64]:
greet()

'HELLO!'

In [73]:
greet

<function __main__.uppercase.<locals>.wrapper()>

### Applying Multiple Decorators to a Function

In [74]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

In [75]:
@strong
@emphasis
def greet():
    return 'Hello!'

In [76]:
greet()

'<strong><em>Hello!</em></strong>'

### Decorating Functions That Accept Arguments

In [79]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() with {args}, {kwargs}')
        original_result = func(*args, **kwargs)
        print(f'TRACE: {func.__name__}() returned {original_result!r}')
        return original_result
    
    return wrapper

In [80]:
@trace
def say(name, line):
    return f'{name}: {line}'

In [83]:
say('Igor', 'Hello, World')

TRACE: calling say() with ('Igor', 'Hello, World'), {}
TRACE: say() returned 'Igor: Hello, World'


'Igor: Hello, World'

### How to Write “Debuggable” Decorators

In [85]:
def greet():
    """Return a friendly greeting."""
    
    return 'Hello!'

decorated_greet = uppercase(greet)

In [86]:
print(greet.__name__)
print(greet.__doc__)

greet
Return a friendly greeting.


In [87]:
print(decorated_greet.__name__)
print(decorated_greet.__doc__)

wrapper
None


In [89]:
def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    
    return wrapper

In [90]:
decorated_greet = uppercase(greet)

In [91]:
print(decorated_greet.__name__)
print(decorated_greet.__doc__)

greet
Return a friendly greeting.


## 3.4 Fun With *args and **kwargs

In [92]:
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)

In [97]:
foo('Hello!')

Hello!


In [98]:
foo('Hello!', 'dispensible_1', 'dispensible_2')

Hello!
('dispensible_1', 'dispensible_2')


In [99]:
foo('Hello!', a=1, b=2)

Hello!
{'a': 1, 'b': 2}


In [96]:
foo('Hello!', 'dispensible_1', 'dispensible_2', a=1, b=2)

Hello!
('dispensible_1', 'dispensible_2')
{'a': 1, 'b': 2}


### Forwarding Optional or Keyword Arguments

In [100]:
def foo(x, *args, **kwargs):
    kwargs['name'] = 'Alice'
    new_args = args + ('extra', )
    bar(x, *new_args, **kwargs)

In [101]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

class AlwaysBlueCar(Car):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.color = 'blue'

In [102]:
AlwaysBlueCar('green', 48392).color

'blue'

In [103]:
def trace(f):
    @functools.wraps(f)
    def decorated_function(*args, **kwargs):
        print(f, args, kwargs)
        result = f(*args, **kwargs)
        print(result)
    return decorated_function

@trace
def greet(greeting, name):
    return '{}, {}!'.format(greeting, name)

In [104]:
greet('Hello', 'Igor')

<function greet at 0x000001B4D67318B0> ('Hello', 'Igor') {}
Hello, Igor!


## 3.5 Function Argument Unpacking

In [105]:
def print_vector(x, y, z):
    print(f'<{x}, {y}, {z}>')

In [106]:
tuple_vec = (1, 0, 1)
list_vec = [1, 0, 1]

In [107]:
print_vector(*tuple_vec)

<1, 0, 1>


In [108]:
print_vector(*list_vec)

<1, 0, 1>


In [109]:
genexpr = (x * x for x in range(3))
print_vector(*genexpr)

<0, 1, 4>


In [110]:
dict_vec = {'y': 0, 'z': 1, 'x': 1}
print_vector(**dict_vec)

<1, 0, 1>


In [111]:
print_vector(*dict_vec)

<y, z, x>


In [114]:
print("Tuple: ", *tuple_vec)
print("List: ", *list_vec)

Tuple:  1 0 1
List:  1 0 1


## 3.6 Nothing to Return Here

In [115]:
def foo1(value):
    if value:
        return value
    else:
        return None

def foo2(value):
    """Bare return statement implies `return None`"""
    if value:
        return value
    else:
        return

def foo3(value):
    """Missing return statement implies `return None`"""
    if value:
        return value

In [119]:
print(foo1(0))

None


In [120]:
print(foo2(0))

None


In [121]:
print(foo3(0))

None
